Whole pipeline working with balanced data and all videos:

In [1]:
# import os
# import sys
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import OneHotEncoder
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import Rescaling, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# from tensorflow.keras.callbacks import EarlyStopping

# df = pd.read_csv('/content/drive/MyDrive/Bundesliga/train.csv')
# df_events = df[~df['event_attributes'].isna()].copy()
# df_pass = df[df['event_attributes'].str[:6] == "['pass"] # Not relevant

# print(f"Accuracy if classifying every class as {df_events['event_attributes'].describe()['top']}: "
#       f"{df_events['event_attributes'].describe()['freq'] / df_events['event_attributes'].describe()['count']:.2%}")
# print(f'Accuracy if classifying every class as a pass (in general) {df_pass.shape[0] / df_events.shape[0]:.2%}')

# df_events.reset_index(drop=True, inplace=True)
# video_ids = df_events['video_id'].unique().tolist()
# train_videos = video_ids[:9]
# test_videos = video_ids[9:]

# X = list()
# for video_id in video_ids:
#   current_video = np.load(f'/content/drive/MyDrive/Bundesliga/PicsFromVids/CompressedPic/{video_id}.npz')
#   current_video = current_video.f.arr_0
#   X.append(current_video)
# X = np.concatenate(X, axis=0)

# train_idx = df_events[df_events['video_id'].isin(train_videos)].index
# test_idx = df_events[df_events['video_id'].isin(test_videos)].index
# X_train = X[train_idx]
# X_test = X[test_idx]
# assert df_events.shape[0] == X_train.shape[0] + X_test.shape[0]

# y_train = df_events[df_events.index.isin(train_idx) ]['event'].to_frame()
# y_test = df_events[df_events.index.isin(test_idx) ]['event'].to_frame()

# play_indexes = y_train[y_train['event'] == 'play'].index
# assert y_train.value_counts()['play'] == len(play_indexes)

# play_amount = y_train.value_counts()['play']
# challenge_amount = y_train.value_counts()['challenge']
# drop_play_indexes = np.sort(np.random.choice(a=play_indexes, 
#                                              size=play_amount - challenge_amount, 
#                                              replace=False))
# assert len(drop_play_indexes) == play_amount - challenge_amount

# y_train.drop(drop_play_indexes, axis=0, inplace=True)
# X_train = np.delete(X_train, drop_play_indexes, axis=0)

# current_feat = ['event']
# for y in (y_train, y_test):
#   encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
#   encoder.fit(y[current_feat]) # Fit and transform data
#   y[encoder.get_feature_names_out(list(current_feat))] = encoder.transform(y[current_feat])
#   y = y.drop(columns=current_feat)

# input_shape = X_train.shape[1], X_train.shape[2], X_train.shape[3]

# model = Sequential([
#     Rescaling(1/255, 
#               input_shape=input_shape),   
#     Conv2D(96, 
#            11, 
#            strides=4, 
#            activation='relu'),
#     MaxPooling2D(pool_size=3, 
#                  strides=2),
#     Conv2D(256, 
#            5, 
#            activation='relu'),
#     MaxPooling2D(pool_size=3, 
#                  strides=2),
#     Conv2D(384, 
#            3, 
#            activation='relu'),
#     Conv2D(384, 
#            3, 
#            activation='relu'),
#     Conv2D(256, 
#            3, 
#            activation='relu'),
#     MaxPooling2D(),
#     Flatten(),
#     Dense(256, 
#           activation='relu'),
#     Dense(256, 
#           activation='relu'),
#     Dense(3, 
#           activation='softmax')
# ])

# model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics='accuracy',
#               )

# callback = EarlyStopping(monitor='val_loss',
#                          patience=3,
#                          restore_best_weights=True
#                          )

# EPOCHS = 100
# model.fit(X_train, 
#           y_train, 
#           validation_split=0.2, 
#           batch_size=16, 
#           epochs=EPOCHS,
#           callbacks=[callback])

# INDEX_ACCURACY = 1
# print(f'\nAccuracy on the test set: {model.evaluate(X_test, y_test)[INDEX_ACCURACY]:.4f}')

# y_pred = model.predict(X_test)
# np.set_printoptions(threshold=sys.maxsize)
# print(y_pred)

Accuracy if classifying every class as ['pass', 'openplay']: 76.15%
Accuracy if classifying every class as a pass (in general) 82.66%


Same procedure as pipeline above:

In [ ]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Rescaling, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
! ls '/content/drive/'

MyDrive


In [4]:
os.chdir('/content/drive/MyDrive/Bundesliga')

In [5]:
! pwd

/content/drive/MyDrive/Bundesliga


In [6]:
# ! unzip dfl-bundesliga-data-shootout.zip

In [7]:
df = pd.read_csv('/content/drive/MyDrive/Bundesliga/train.csv')

In [8]:
# df['time_ms'] = round(df['time'] * 1000).astype(int)

In [9]:
display(df.head())
df.shape

,video_id,time,event,event_attributes,time_ms
0,1606b0e6_0,200.265822,start,NaN,200266
1,1606b0e6_0,201.150000,challenge,['ball_action_forced'],201150
2,1606b0e6_0,202.765822,end,NaN,202766
3,1606b0e6_0,210.124111,start,NaN,210124
4,1606b0e6_0,210.870000,challenge,['opponent_dispossessed'],210870


(11218, 5)

In [10]:
df_events = df[~df['event_attributes'].isna()].copy()

In [11]:
# df_events['frame_id'] = df_events['video_id'] + df_events['time_ms'].astype(str)

In [12]:
# # Reordering columns
# df_events = df_events[['video_id', 'frame_id', 'time', 
#                        'time_ms', 'event', 'event_attributes']]

In [13]:
display(df_events.head())
df_events.shape

,video_id,frame_id,time,time_ms,event,event_attributes
1,1606b0e6_0,1606b0e6_0201150,201.15,201150,challenge,['ball_action_forced']
4,1606b0e6_0,1606b0e6_0210870,210.87,210870,challenge,['opponent_dispossessed']
7,1606b0e6_0,1606b0e6_0219230,219.23,219230,throwin,['pass']
10,1606b0e6_0,1606b0e6_0224430,224.43,224430,play,"['pass', 'openplay']"
13,1606b0e6_0,1606b0e6_0229390,229.39,229390,play,"['pass', 'openplay']"


(4382, 6)

Building very basic baseline model:

In [14]:
df_pass = df[df['event_attributes'].str[:6] == "['pass"]

In [15]:
display(df_pass.head())
df_pass.shape

,video_id,time,event,event_attributes,time_ms
7,1606b0e6_0,219.23,throwin,['pass'],219230
10,1606b0e6_0,224.43,play,"['pass', 'openplay']",224430
13,1606b0e6_0,229.39,play,"['pass', 'openplay']",229390
16,1606b0e6_0,236.71,play,"['pass', 'openplay']",236710
17,1606b0e6_0,239.35,play,"['pass', 'openplay']",239350


(3622, 5)

In [16]:
df_events['event_attributes'].value_counts()

['pass', 'openplay']                  3337
['ball_action_forced']                 239
['pass']                               154
['opponent_dispossessed']              138
['pass', 'freekick']                   127
['fouled']                             111
['cross', 'openplay']                   80
['challenge_during_ball_transfer']      53
['possession_retained']                 44
['opponent_rounded']                    39
['cross', 'corner']                     33
['cross']                               18
['cross', 'freekick']                    5
['pass', 'corner']                       4
Name: event_attributes, dtype: int64

In [17]:
print(f"Accuracy if classifying every class as {df_events['event_attributes'].describe()['top']}: "
      f"{df_events['event_attributes'].describe()['freq'] / df_events['event_attributes'].describe()['count']:.2%}")

Accuracy if classifying every class as ['pass', 'openplay']: 76.15%


In [18]:
print(f'Accuracy if classifying every class as a pass (in general) {df_pass.shape[0] / df_events.shape[0]:.2%}')

Accuracy if classifying every class as a pass (in general) 82.66%


In [19]:
df_events['event'].value_counts()

play         3586
challenge     624
throwin       172
Name: event, dtype: int64

In [20]:
df_video1 = df_events[df_events['video_id'] == '1606b0e6_0']
df_video1

,video_id,frame_id,time,time_ms,event,event_attributes
1,1606b0e6_0,1606b0e6_0201150,201.15,201150,challenge,['ball_action_forced']
4,1606b0e6_0,1606b0e6_0210870,210.87,210870,challenge,['opponent_dispossessed']
7,1606b0e6_0,1606b0e6_0219230,219.23,219230,throwin,['pass']
10,1606b0e6_0,1606b0e6_0224430,224.43,224430,play,"['pass', 'openplay']"
13,1606b0e6_0,1606b0e6_0229390,229.39,229390,play,"['pass', 'openplay']"
...,...,...,...,...,...,...
986,1606b0e6_0,1606b0e6_02861430,2861.43,2861430,play,"['pass', 'openplay']"
989,1606b0e6_0,1606b0e6_02900110,2900.11,2900110,challenge,['ball_action_forced']
992,1606b0e6_0,1606b0e6_02903430,2903.43,2903430,challenge,['fouled']
995,1606b0e6_0,1606b0e6_03000870,3000.87,3000870,play,"['pass', 'freekick']"


In [21]:
df_video2 = df_events[df_events['video_id'] == '1606b0e6_1']
df_video2

,video_id,frame_id,time,time_ms,event,event_attributes
1001,1606b0e6_1,1606b0e6_1510590,510.59,510590,play,"['pass', 'openplay']"
1002,1606b0e6_1,1606b0e6_1512470,512.47,512470,play,"['pass', 'openplay']"
1005,1606b0e6_1,1606b0e6_1517070,517.07,517070,play,"['pass', 'openplay']"
1006,1606b0e6_1,1606b0e6_1518430,518.43,518430,play,"['pass', 'openplay']"
1007,1606b0e6_1,1606b0e6_1520990,520.99,520990,play,"['pass', 'openplay']"
...,...,...,...,...,...,...
2235,1606b0e6_1,1606b0e6_13357990,3357.99,3357990,challenge,['fouled']
2238,1606b0e6_1,1606b0e6_13364870,3364.87,3364870,play,"['pass', 'freekick']"
2241,1606b0e6_1,1606b0e6_13373790,3373.79,3373790,play,"['pass', 'openplay']"
2244,1606b0e6_1,1606b0e6_13376550,3376.55,3376550,play,"['pass', 'openplay']"


In [22]:
y_train_val = df_video1['event']
y_train_val.to_frame()

,event
1,challenge
4,challenge
7,throwin
10,play
13,play
...,...
986,play
989,challenge
992,challenge
995,play


In [23]:
y_test = df_video2['event']
y_test.to_frame()

,event
1001,play
1002,play
1005,play
1006,play
1007,play
...,...
2235,challenge
2238,play
2241,play
2244,play


In [24]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

current_feat = ['event']

print('Shape of y_test before transformation: ', y_test.shape)
y_test = y_test.to_frame()

# Fit and transform data
encoder.fit(y_test[current_feat])
y_test[encoder.get_feature_names_out(list(current_feat))] = encoder.transform(y_test[current_feat])
y_test = y_test.drop(columns=current_feat)

print('Shape of y_test after transformation: ', y_test.shape)

Shape of y_test before transformation:  (507,)
Shape of y_test after transformation:  (507, 3)


In [25]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

current_feat = ['event']

print('Shape of y_train_val before transformation: ', y_train_val.shape)
y_train_val = y_train_val.to_frame()

# Fit and transform data
encoder.fit(y_train_val[current_feat])
y_train_val[encoder.get_feature_names_out(list(current_feat))] = encoder.transform(y_train_val[current_feat])
y_train_val = y_train_val.drop(columns=current_feat)

print('Shape of y_train_val after transformation: ', y_train_val.shape)

Shape of y_train_val before transformation:  (396,)
Shape of y_train_val after transformation:  (396, 3)


Files without compressing:

In [23]:
# X_train = np.load('/content/drive/MyDrive/Bundesliga/PicsFromVids/1606b0e6_0.npy')
# X_train2 = np.load('/content/drive/MyDrive/Bundesliga/PicsFromVids/1606b0e6_1.npy')
# X_train3 = np.load('/content/drive/MyDrive/Bundesliga/PicsFromVids/35bd9041_0.npy')
# X_train4 = np.load('/content/drive/MyDrive/Bundesliga/PicsFromVids/35bd9041_1.npy')
# X_train5 = np.load('/content/drive/MyDrive/Bundesliga/PicsFromVids/3c993bd2_0.npy')

In [24]:
# X_train.shape
# X_train2.shape, X_train3.shape, X_train4.shape, X_train5.shape

In [25]:
# X_train.max()
# X_train2.max(), X_train3.max(), X_train4.max(), X_train5.max()

# Working with compressed pictures:

## Training the model with just the first video and original (unbalanced) data:

In [26]:
X_train_val = np.load('/content/drive/MyDrive/Bundesliga/PicsFromVids/CompressedPic/1606b0e6_0.npz')

In [40]:
X_test = np.load('/content/drive/MyDrive/Bundesliga/PicsFromVids/CompressedPic/1606b0e6_1.npz')
X_test = X_test.f.arr_0

In [ ]:
X_train_val = X_train_val.f.arr_0

In [27]:
X_train_val.f.arr_0.shape

(396, 480, 640, 3)

In [41]:
X_test.shape

(507, 480, 640, 3)

In [31]:
input_shape = X_train_val.shape[1], X_train_val.shape[2], X_train_val.shape[3]

model = Sequential([
    Rescaling(1/255, 
              input_shape=input_shape),   
    Conv2D(96, 
           11, 
           strides=4, 
           activation='relu'),
    MaxPooling2D(pool_size=3, 
                 strides=2),
    Conv2D(256, 
           5, 
           activation='relu'),
    MaxPooling2D(pool_size=3, 
                 strides=2),
    Conv2D(384, 
           3, 
           activation='relu'),
    Conv2D(384, 
           3, 
           activation='relu'),
    Conv2D(256, 
           3, 
           activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(256, 
          activation='relu'),
    Dense(256, 
          activation='relu'),
    Dense(3, 
          activation='softmax')
])

In [32]:
model.input_shape, model.output_shape

((None, 480, 640, 3), (None, 3))

In [33]:
X_train_val.shape, y_train_val.shape

((396, 480, 640, 3), (396, 3))

In [34]:
y_train_val.value_counts()

event_challenge  event_play  event_throwin
0.0              1.0         0.0              319
1.0              0.0         0.0               56
0.0              0.0         1.0               21
dtype: int64

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 480, 640, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 118, 158, 96)      34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 58, 78, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 54, 74, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 26, 36, 256)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 34, 384)       8

In [36]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics='accuracy',
              )

In [37]:
callback = EarlyStopping(monitor='val_loss',
                         patience=5,
                         restore_best_weights=True
                         )

In [38]:
EPOCHS = 100
model.fit(X_train_val, 
          y_train_val, 
          validation_split=0.2, 
          batch_size=16, 
          epochs=EPOCHS,
          callbacks=[callback])

Epoch 1/100
20/20 [==============================] - 143s 7s/step - loss: 1.0201 - accuracy: 0.7310 - val_loss: 0.6056 - val_accuracy: 0.8500
Epoch 2/100
20/20 [==============================] - 134s 7s/step - loss: 0.6940 - accuracy: 0.7943 - val_loss: 0.5458 - val_accuracy: 0.8500
Epoch 3/100
20/20 [==============================] - 134s 7s/step - loss: 0.6615 - accuracy: 0.7943 - val_loss: 0.5162 - val_accuracy: 0.8500
Epoch 4/100
20/20 [==============================] - 134s 7s/step - loss: 0.6504 - accuracy: 0.7943 - val_loss: 0.5096 - val_accuracy: 0.8500
Epoch 5/100
20/20 [==============================] - 134s 7s/step - loss: 0.6485 - accuracy: 0.7943 - val_loss: 0.5485 - val_accuracy: 0.8500
Epoch 6/100
20/20 [==============================] - 134s 7s/step - loss: 0.6371 - accuracy: 0.7943 - val_loss: 0.5258 - val_accuracy: 0.8500
Epoch 7/100
20/20 [==============================] - 133s 7s/step - loss: 0.6396 - accuracy: 0.7943 - val_loss: 0.6021 - val_accuracy: 0.8500
Epoch 

In [48]:
INDEX_ACCURACY = 1
print(f'\nAccuracy on the test set: {model.evaluate(X_test, y_test)[INDEX_ACCURACY]:.4f}')

16/16 [==============================] - 63s 4s/step - loss: 0.4800 - accuracy: 0.8560

Accuracy on the test set: 0.8560


In [49]:
y_pred = model.predict(X_test)

16/16 [==============================] - 62s 4s/step


In [51]:
import sys
np.set_printoptions(threshold=sys.maxsize)
y_pred

array([[0.127561  , 0.8448479 , 0.0275911 ],
       [0.12767003, 0.8446903 , 0.02763961],
       [0.12760173, 0.8447908 , 0.0276075 ],
       [0.1273588 , 0.84514177, 0.0274994 ],
       [0.127394  , 0.8450924 , 0.0275136 ],
       [0.12755343, 0.84486175, 0.0275849 ],
       [0.12739739, 0.84508526, 0.02751737],
       [0.12758857, 0.84480816, 0.02760324],
       [0.12744667, 0.84501505, 0.02753821],
       [0.12767054, 0.8446902 , 0.02763913],
       [0.127578  , 0.84482336, 0.02759872],
       [0.12768416, 0.8446706 , 0.02764536],
       [0.12764794, 0.84472364, 0.02762833],
       [0.1275285 , 0.844897  , 0.02757455],
       [0.12746635, 0.8449872 , 0.02754644],
       [0.12762552, 0.84475535, 0.02761909],
       [0.12758239, 0.8448178 , 0.02759966],
       [0.12763423, 0.84474236, 0.02762334],
       [0.12738717, 0.8450998 , 0.02751304],
       [0.12738664, 0.8451025 , 0.02751088],
       [0.12742066, 0.8450523 , 0.02752698],
       [0.1275657 , 0.8448426 , 0.02759168],
       [0.

## Training the model with all videos and balanced (undersampling) data:

In [26]:
df_events

,video_id,frame_id,time,time_ms,event,event_attributes
1,1606b0e6_0,1606b0e6_0201150,201.150,201150,challenge,['ball_action_forced']
4,1606b0e6_0,1606b0e6_0210870,210.870,210870,challenge,['opponent_dispossessed']
7,1606b0e6_0,1606b0e6_0219230,219.230,219230,throwin,['pass']
10,1606b0e6_0,1606b0e6_0224430,224.430,224430,play,"['pass', 'openplay']"
13,1606b0e6_0,1606b0e6_0229390,229.390,229390,play,"['pass', 'openplay']"
...,...,...,...,...,...,...
11206,ecf251d4_0,ecf251d4_03041347,3041.347,3041347,play,"['pass', 'openplay']"
11209,ecf251d4_0,ecf251d4_03050347,3050.347,3050347,play,"['pass', 'openplay']"
11210,ecf251d4_0,ecf251d4_03053067,3053.067,3053067,play,"['pass', 'openplay']"
11213,ecf251d4_0,ecf251d4_03056587,3056.587,3056587,challenge,['opponent_dispossessed']


In [27]:
df_events.reset_index(drop=True, inplace=True)
df_events

,video_id,frame_id,time,time_ms,event,event_attributes
0,1606b0e6_0,1606b0e6_0201150,201.150,201150,challenge,['ball_action_forced']
1,1606b0e6_0,1606b0e6_0210870,210.870,210870,challenge,['opponent_dispossessed']
2,1606b0e6_0,1606b0e6_0219230,219.230,219230,throwin,['pass']
3,1606b0e6_0,1606b0e6_0224430,224.430,224430,play,"['pass', 'openplay']"
4,1606b0e6_0,1606b0e6_0229390,229.390,229390,play,"['pass', 'openplay']"
...,...,...,...,...,...,...
4377,ecf251d4_0,ecf251d4_03041347,3041.347,3041347,play,"['pass', 'openplay']"
4378,ecf251d4_0,ecf251d4_03050347,3050.347,3050347,play,"['pass', 'openplay']"
4379,ecf251d4_0,ecf251d4_03053067,3053.067,3053067,play,"['pass', 'openplay']"
4380,ecf251d4_0,ecf251d4_03056587,3056.587,3056587,challenge,['opponent_dispossessed']


In [65]:
video_ids = df_events['video_id'].unique().tolist()
train_videos = video_ids[:9]
test_videos = video_ids[9:]

In [29]:
train_videos

['1606b0e6_0',
 '1606b0e6_1',
 '35bd9041_0',
 '35bd9041_1',
 '3c993bd2_0',
 '3c993bd2_1',
 '407c5a9e_1',
 '4ffd5986_0',
 '9a97dae4_1']

In [30]:
test_videos

['cfbe2e94_0', 'cfbe2e94_1', 'ecf251d4_0']

In [31]:
X = list()
for video_id in video_ids:
  current_video = np.load(f'/content/drive/MyDrive/Bundesliga/PicsFromVids/CompressedPic/{video_id}.npz')
  current_video = current_video.f.arr_0
  X.append(current_video)

In [32]:
X = np.concatenate(X, axis=0)

In [33]:
X.shape

(4382, 480, 640, 3)

In [34]:
train_idx = df_events[df_events['video_id'].isin(train_videos)].index
test_idx = df_events[df_events['video_id'].isin(test_videos)].index

In [44]:
train_idx

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            3396, 3397, 3398, 3399, 3400, 3401, 3402, 3403, 3404, 3405],
           dtype='int64', length=3406)

In [35]:
X_train = X[train_idx]
X_test = X[test_idx]

In [36]:
X_train.shape

(3406, 480, 640, 3)

In [37]:
X_test.shape

(976, 480, 640, 3)

In [38]:
assert df_events.shape[0] == X_train.shape[0] + X_test.shape[0]

In [53]:
y_train = df_events[df_events.index.isin(train_idx) ]['event'].to_frame()
y_test = df_events[df_events.index.isin(test_idx) ]['event'].to_frame()

In [54]:
y_train.value_counts()

event    
play         2789
challenge     505
throwin       112
dtype: int64

In [55]:
play_indexes = y_train[y_train['event'] == 'play'].index

In [56]:
assert y_train.value_counts()['play'] == len(play_indexes)

In [57]:
play_amount = y_train.value_counts()['play']
challenge_amount = y_train.value_counts()['challenge']
challenge_amount

505

In [58]:
drop_play_indexes = np.sort(np.random.choice(a=play_indexes, 
                                             size=play_amount - challenge_amount, 
                                             replace=False))
drop_play_indexes[-10:]

array([3387, 3389, 3390, 3392, 3393, 3394, 3395, 3397, 3399, 3402])

In [59]:
assert len(drop_play_indexes) == play_amount - challenge_amount

Deleting rows to perform and undersampling in the class 'play':

In [60]:
y_train.drop(drop_play_indexes, axis=0, inplace=True)
y_train.shape

(1122, 1)

In [61]:
y_train.value_counts()

event    
challenge    505
play         505
throwin      112
dtype: int64

In [62]:
X_train.shape

(3406, 480, 640, 3)

Deleting the same rows from X_train:

In [63]:
X_train = np.delete(X_train, drop_play_indexes, axis=0)

In [64]:
X_train.shape

(1122, 480, 640, 3)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [175]:
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

current_feat = ['event']

print(f'Shape of y_train before transformation: ', y_train.shape)

# Fit and transform data
encoder.fit(y_train[current_feat])
y_train[encoder.get_feature_names_out(list(current_feat))] = encoder.transform(y_train[current_feat])
y_train = y_train.drop(columns=current_feat)

print(f'Shape of y_train after transformation: ', y_train.shape)

Shape of y_train before transformation:  (1122, 4)
Shape of y_train after transformation:  (1122, 3)


In [177]:
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

current_feat = ['event']

print(f'Shape of y_train before transformation: ', y_test.shape)

# Fit and transform data
encoder.fit(y_test[current_feat])
y_test[encoder.get_feature_names_out(list(current_feat))] = encoder.transform(y_test[current_feat])
y_test = y_test.drop(columns=current_feat)

print(f'Shape of y_test after transformation: ', y_test.shape)

Shape of y_train before transformation:  (976, 4)
Shape of y_test after transformation:  (976, 3)


In [179]:
input_shape = X_train.shape[1], X_train.shape[2], X_train.shape[3]

model = Sequential([
    Rescaling(1/255, 
              input_shape=input_shape),   
    Conv2D(96, 
           11, 
           strides=4, 
           activation='relu'),
    MaxPooling2D(pool_size=3, 
                 strides=2),
    Conv2D(256, 
           5, 
           activation='relu'),
    MaxPooling2D(pool_size=3, 
                 strides=2),
    Conv2D(384, 
           3, 
           activation='relu'),
    Conv2D(384, 
           3, 
           activation='relu'),
    Conv2D(256, 
           3, 
           activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(256, 
          activation='relu'),
    Dense(256, 
          activation='relu'),
    Dense(3, 
          activation='softmax')
])

In [180]:
model.input_shape, model.output_shape

((None, 480, 640, 3), (None, 3))

In [181]:
X_train.shape, y_train.shape

((1122, 480, 640, 3), (1122, 3))

In [182]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 480, 640, 3)       0         
                                                                 
 conv2d_5 (Conv2D)           (None, 118, 158, 96)      34944     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 58, 78, 96)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 54, 74, 256)       614656    
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 26, 36, 256)      0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 24, 34, 384)      

In [183]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics='accuracy',
              )

In [185]:
callback = EarlyStopping(monitor='val_loss',
                         patience=3,
                         restore_best_weights=True
                         )

In [186]:
EPOCHS = 100
model.fit(X_train, 
          y_train, 
          validation_split=0.2, 
          batch_size=16, 
          epochs=EPOCHS,
          callbacks=[callback])

Epoch 1/100
57/57 [==============================] - 386s 7s/step - loss: 1.0590 - accuracy: 0.4337 - val_loss: 0.9371 - val_accuracy: 0.3911
Epoch 2/100
57/57 [==============================] - 381s 7s/step - loss: 0.9694 - accuracy: 0.4392 - val_loss: 0.9425 - val_accuracy: 0.5289
Epoch 3/100
57/57 [==============================] - 382s 7s/step - loss: 0.9679 - accuracy: 0.4281 - val_loss: 0.9163 - val_accuracy: 0.5289
Epoch 4/100
57/57 [==============================] - 382s 7s/step - loss: 0.9647 - accuracy: 0.4582 - val_loss: 0.9125 - val_accuracy: 0.5289
Epoch 5/100
57/57 [==============================] - 383s 7s/step - loss: 0.9622 - accuracy: 0.4537 - val_loss: 0.9263 - val_accuracy: 0.3911
Epoch 6/100
57/57 [==============================] - 382s 7s/step - loss: 0.9593 - accuracy: 0.4571 - val_loss: 0.9235 - val_accuracy: 0.5289
Epoch 7/100
57/57 [==============================] - 382s 7s/step - loss: 0.9601 - accuracy: 0.4649 - val_loss: 0.9333 - val_accuracy: 0.3911


In [187]:
INDEX_ACCURACY = 1
print(f'\nAccuracy on the test set: {model.evaluate(X_test, y_test)[INDEX_ACCURACY]:.4f}')

31/31 [==============================] - 121s 4s/step - loss: 0.9561 - accuracy: 0.1219

Accuracy on the test set: 0.1219


In the test set the data is unbalanced and that's why we see such a difference between the validation and the test set.

In [188]:
y_pred = model.predict(X_test)

31/31 [==============================] - 119s 4s/step


In [189]:
import sys
np.set_printoptions(threshold=sys.maxsize)
y_pred

array([[0.48357105, 0.40877905, 0.10764986],
       [0.48357168, 0.40877914, 0.10764921],
       [0.4835699 , 0.40877905, 0.10765103],
       [0.48356897, 0.40877903, 0.10765199],
       [0.4835689 , 0.40877905, 0.10765209],
       [0.4835689 , 0.40877905, 0.1076521 ],
       [0.4835689 , 0.40877905, 0.1076521 ],
       [0.48357075, 0.40877908, 0.10765021],
       [0.4835692 , 0.40877905, 0.10765181],
       [0.48356944, 0.40877905, 0.10765151],
       [0.48356897, 0.40877905, 0.10765204],
       [0.48356932, 0.40877905, 0.10765161],
       [0.48356932, 0.40877905, 0.10765161],
       [0.4835692 , 0.40877905, 0.10765179],
       [0.48356938, 0.40877903, 0.10765156],
       [0.48356956, 0.408779  , 0.10765141],
       [0.4835705 , 0.4087791 , 0.10765041],
       [0.4835699 , 0.40877905, 0.10765106],
       [0.48357105, 0.40877905, 0.10764983],
       [0.48356932, 0.40877903, 0.1076516 ],
       [0.4835695 , 0.40877903, 0.10765149],
       [0.48357   , 0.40877905, 0.10765095],
       [0.